In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [4]:
url = 'https://merolagani.com/CompanyDetail.aspx?symbol=GBIME#0'

In [5]:
response = requests.get(url)
response

<Response [200]>

In [6]:
response = response.content

In [7]:
soup = BeautifulSoup(response, 'html.parser')

In [8]:
soup


<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml">
<head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1" name="viewport"/><title>
	merolagani - Global IME Bank Limited (GBIME) Detail, Announcements, News, Price History, Floorsheet
</title>
<link as="style" href="/bundles/css/main200?v=XBmv3Nbj66DV6lzAxQCBgaIOrn903Diuut8lsSb3n9k1" rel="preload"/>
<link as="style" href="/bundles/css/site205?v=L5V1uk5xoLic8qHQPSOAgTWwYTf5257cWEew3F97Y_c1" rel="preload"/>
<link href="/bundles/css/main200?v=XBmv3Nbj66DV6lzAxQCBgaIOrn903Diuut8lsSb3n9k1" rel="stylesheet">
<link href="/bundles/css/site205?v=L5V1uk5xoLic8qHQPSOAgTWwYTf5257cWEew3F97Y_c1" rel="stylesheet">
<link href="/Content/Services.css" rel="stylesheet"/>
<link href="/Content/owl.carousel.min.css" rel="stylesheet"/>
<link href="/Content/lightslider/css/lightslider.css" rel="stylesheet"/>
<script src="/scripts/jquery/jquery-1.11.0.min.js"></script>
<scr

In [9]:
table= soup.find_all('table', class_='table table-bordered table-striped table-hover')

### Best Code

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, UnexpectedAlertPresentException
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import os

# Create company directory if it doesn't exist
if not os.path.exists('company'):
    os.makedirs('company')

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-notifications")  # Disable notifications
chrome_options.add_argument("--disable-popup-blocking")  # Disable popup blocking
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.notifications": 2  # Block notifications
})

symbols_list = [
    # 'IGI'  # Add more symbols as needed
    'LEC', 'MLBL', 'SRLI', "GFCL", "CITY"
]

# Service for ChromeDriver
service = Service(r"C:\Users\baral\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")

def handle_alert(driver):
    try:
        alert = driver.switch_to.alert
        alert.dismiss()
    except:
        pass

for symbol in symbols_list:
    url = f"https://merolagani.com/CompanyDetail.aspx?symbol={symbol}"
    share_name = url.split("=")[-1]
    all_data = []
    
    try:
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.get(url)
        
        # Handle any initial alerts
        handle_alert(driver)
        
        # Wait and click on the price history tab
        try:
            price_history_tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, 'ctl00_ContentPlaceHolder1_CompanyDetail1_lnkHistoryTab'))
            )
            price_history_tab.click()
            time.sleep(3)  # Wait for content to load
            
            # Collect data from the first page
            handle_alert(driver)
            table_present = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "table-bordered"))
            )
            html_content = driver.page_source
            soup = BeautifulSoup(html_content, "html.parser")
            table = soup.find("table", class_="table-bordered")
            if table:
                each_row = table.find_all('tr')
                for row in each_row[1:]:  # Skip header row
                    td = row.find_all('td')
                    row_data = [cell.text.strip() for cell in td if cell.text.strip()]
                    if row_data:
                        all_data.append(row_data)
            
            # Navigate through pagination
            while True:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, '//a[@title="Next Page"]'))
                    )
                    next_button.click()
                    time.sleep(2)  # Wait between page loads
                    
                    # Handle any alerts after clicking next page
                    handle_alert(driver)
                    
                    # Wait for table to load on the new page
                    table_present = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "table-bordered"))
                    )
                    
                    # Collect data from the new page
                    html_content = driver.page_source
                    soup = BeautifulSoup(html_content, "html.parser")
                    table = soup.find("table", class_="table-bordered")
                    if table:
                        each_row = table.find_all('tr')
                        for row in each_row[1:]:  # Skip header row
                            td = row.find_all('td')
                            row_data = [cell.text.strip() for cell in td if cell.text.strip()]
                            if row_data:
                                all_data.append(row_data)
                except TimeoutException:
                    break  # No more pages to navigate
                except UnexpectedAlertPresentException:
                    handle_alert(driver)
                    break  # Handle alert and exit loop if necessary
                
        except TimeoutException:
            print(f"Timeout waiting for elements on {share_name}")
        except UnexpectedAlertPresentException:
            handle_alert(driver)
            
    except Exception as e:
        print(f"An error occurred for {share_name}: {str(e)}")
        
    finally:
        try:
            driver.quit()
        except:
            pass
        
    # Save data to CSV
    if all_data:
        df = pd.DataFrame(all_data, columns=["SN", "Date", "LTP", "%change", "High", "Low", "Open", "Qty.", "Turnover"])
        df.to_csv(f'company/{share_name}.csv', index=False)
        print(f"Successfully saved data for {share_name}")
    else:
        print(f"No data found for {share_name}")

O1 answer

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, UnexpectedAlertPresentException
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import os

# Create company directory if it doesn't exist
if not os.path.exists('company'):
    os.makedirs('company')

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.notifications": 2
})

symbols_list = ['IGI','LEC','MLBL','SRLI','GFCL','CITY']
service = Service(r"C:\Users\baral\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")

def handle_alert(driver):
    try:
        alert = driver.switch_to.alert
        alert.dismiss()
    except:
        pass

def close_notification_popup(driver):
    try:
        deny_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@id="onesignal-slidedown-cancel-button"]'))
        )
        deny_button.click()
    except:
        pass

for symbol in symbols_list:
    url = f"https://merolagani.com/CompanyDetail.aspx?symbol={symbol}"
    share_name = url.split("=")[-1]
    all_data = []

    try:
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.get(url)

        handle_alert(driver)
        close_notification_popup(driver)

        try:
            # Waiting longer for history tab
            # Waiting for the Price History Tab to Be Clickable
            price_history_tab = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.ID, 'ctl00_ContentPlaceHolder1_CompanyDetail1_lnkHistoryTab'))
            )
            price_history_tab.click()
            time.sleep(2)
            handle_alert(driver)
            close_notification_popup(driver)

            # Wait for table presence
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "table-bordered"))
            )
            time.sleep(2)

            soup = BeautifulSoup(driver.page_source, "html.parser")
            table = soup.find("table", class_="table-bordered")
            if table:
                rows = table.find_all('tr')[1:]
                for row in rows:
                    cells = [c.text.strip() for c in row.find_all('td')]
                    if len(cells) == 9:
                        all_data.append(cells)

            # Navigate through pagination
            while True:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, '//a[@title="Next Page"]'))
                    )
                    next_button.click()
                    time.sleep(2)

                    handle_alert(driver)
                    close_notification_popup(driver)

                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "table-bordered"))
                    )
                    soup = BeautifulSoup(driver.page_source, "html.parser")
                    table = soup.find("table", class_="table-bordered")
                    if table:
                        rows = table.find_all('tr')[1:]
                        for row in rows:
                            cells = [c.text.strip() for c in row.find_all('td')]
                            if len(cells) == 9:
                                all_data.append(cells)
                except TimeoutException:
                    break
                except UnexpectedAlertPresentException:
                    handle_alert(driver)
                    break

        except TimeoutException:
            print(f"Timeout waiting for elements on {share_name}")
        except UnexpectedAlertPresentException:
            handle_alert(driver)

    except Exception as e:
        print(f"An error occurred for {share_name}: {str(e)}")
    finally:
        try:
            driver.quit()
        except:
            pass

    if all_data:
        df = pd.DataFrame(all_data, columns=["SN", "Date", "LTP", "%change", "High", "Low", "Open", "Qty.", "Turnover"])
        df.to_csv(f'company/{share_name}.csv', index=False)
        print(f"Successfully saved data for {share_name}")
    else:
        print(f"No data found for {share_name}")

Successfully saved data for IGI
Successfully saved data for LEC
Successfully saved data for MLBL
Successfully saved data for SRLI
Successfully saved data for GFCL
Successfully saved data for CITY


Changes in 01 code

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, UnexpectedAlertPresentException
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import os

# Create company directory if it doesn't exist
if not os.path.exists('company'):
    os.makedirs('company')

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.notifications": 2
})

symbols_list = ["HDL"]
service = Service(r"C:\Users\baral\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")

def handle_alert(driver):
    try:
        alert = driver.switch_to.alert
        alert.dismiss()
    except:
        pass

def close_notification_popup(driver):
    try:
        deny_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@id="onesignal-slidedown-cancel-button"]'))
        )
        deny_button.click()
    except:
        pass

for symbol in symbols_list:
    url = f"https://merolagani.com/CompanyDetail.aspx?symbol={symbol}"
    share_name = url.split("=")[-1]
    all_data = []

    try:
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.get(url)

        handle_alert(driver)
        close_notification_popup(driver)

        try:
            # Waiting longer for history tab
            # Waiting for the Price History Tab to Be Clickable
            price_history_tab = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.ID, 'ctl00_ContentPlaceHolder1_CompanyDetail1_lnkHistoryTab'))
            )
            price_history_tab.click()
            # time.sleep(2)
            handle_alert(driver)
            close_notification_popup(driver)

            # Wait for table presence
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, "table-bordered"))
            )
            # time.sleep(2)

            soup = BeautifulSoup(driver.page_source, "html.parser")
            table = soup.find("table", class_="table-bordered")
            if table:
                rows = table.find_all('tr')[1:]
                for row in rows:
                    cells = [c.text.strip() for c in row.find_all('td')]
                    if len(cells) == 9:
                        all_data.append(cells)

            # Navigate through pagination
            while True:
                try:
                    next_button = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((By.XPATH, '//a[@title="Next Page"]'))
                    )
                    next_button.click()
                    time.sleep(2)

                    handle_alert(driver)
                    close_notification_popup(driver)

                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "table-bordered"))
                    )
                    soup = BeautifulSoup(driver.page_source, "html.parser")
                    table = soup.find("table", class_="table-bordered")
                    if table:
                        rows = table.find_all('tr')[1:]
                        for row in rows:
                            cells = [c.text.strip() for c in row.find_all('td')]
                            if len(cells) == 9:
                                all_data.append(cells)
                except TimeoutException:
                    break
                except UnexpectedAlertPresentException:
                    handle_alert(driver)
                    break

        except TimeoutException:
            print(f"Timeout waiting for elements on {share_name}")
        except UnexpectedAlertPresentException:
            handle_alert(driver)

    except Exception as e:
        print(f"An error occurred for {share_name}: {str(e)}")
    finally:
        try:
            driver.quit()
        except:
            pass

    if all_data:
        df = pd.DataFrame(all_data, columns=["SN", "Date", "LTP", "%change", "High", "Low", "Open", "Qty.", "Turnover"])
        df.to_csv(f'company/{share_name}.csv', index=False)
        print(f"Successfully saved data for {share_name}")
    else:
        print(f"No data found for {share_name}")

Successfully saved data for HDL


In [9]:
# this is used to use this variable in the share_price.ipynb file
%store share_name


Stored 'share_name' (str)


In [10]:
share_name

'SWMF'

In [11]:
# import numpy as np
# np.shape(all_data)


In [12]:

# # Convert the collected data to a DataFrame
# df = pd.DataFrame(all_data, columns=["SN","Date", "LTP", "%change", "High", "Low", "Open","Qty.", "Turnover"])

# # Convert the DataFrame to a CSV file
# df.to_csv("stock_price.csv", index=False)